# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [5]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [9]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [7]:
 def get_book_data(book_url: str) -> dict:
    """
    МЕСТО ДЛЯ ДОКУМЕНТАЦИИ
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
import requests
from bs4 import BeautifulSoup
def get_book_data(book_url: str) -> dict:
    soup = BeautifulSoup(requests.get(book_url).content, 'html.parser')

    # Основная информация
    name = soup.find('h1').text.strip()

    # Рейтинг
    star_rating_element = soup.find('p', class_=lambda x: x and 'star-rating' in x)
    rating = None
    if star_rating_element:
        classes = star_rating_element.get('class', [])
        rating = next((cls for cls in classes if cls != 'star-rating'), None)

    # Таблица с деталями
    table = soup.find('table', attrs={'class': 'table table-striped'})
    product_data = {}

    for tr in table.find_all('tr'):
        tds = tr.find_all('td')
        th = tr.find('th')
        if th and tds:
            key = th.text.strip().lower().replace(' ', '_').replace('(', '').replace(')', '')
            value = tds[0].text.strip()
            product_data[key] = value

    # Описание
    product_description_elem = soup.find('div', id='product_description')
    product_description = ""
    if product_description_elem:
        product_description = product_description_elem.find_next_sibling('p').text.strip()

    # Объединяем все в один плоский словарь
    return {
        'name': name,
        'rating': rating,
        'description': product_description,
        **product_data  # распаковываем словарь с деталями
    }
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

In [8]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'name': 'A Light in the Attic',
 'rating': 'Three',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up there,And your cradle, too?Baby,

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [11]:
def scrape_books(is_save=False):
    """
    Парсит все книги с сайта books.toscrape.com.

    Проходит по всем страницам каталога, извлекает ссылки на книги и собирает
    полную информацию о каждой книге с помощью функции get_book_data.
    """

    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    # Встроенная функция для получения количества страниц
    def get_total_pages(base_url: str = "http://books.toscrape.com/catalogue/") -> int:
        page = 1
        start_time = time.time()
        print("Начинаю поиск количества страниц...")

        while True:
            url = f"{base_url}page-{page}.html"
            print(f"Страница {page}: ", end="")

            response = requests.get(url)
            if response.status_code != 200:
                print(f"не существует")
                break

            print(f"существует")
            page += 1

        total_pages = page - 1
        end_time = time.time()
        print(f"Поиск завершен за {end_time - start_time:.2f} секунд")
        print(f"Всего страниц: {total_pages}")
        return total_pages

    # Получаем общее количество страниц
    total_pages = get_total_pages()

    all_books_data = []

    def scrape_page(page_num):
        """Парсит одну страницу и возвращает данные книг"""
        link = f"http://books.toscrape.com/catalogue/page-{page_num}.html"
        soup = BeautifulSoup(requests.get(link).content, "html.parser")
        books = soup.find_all('article', class_='product_pod')

        page_books_data = []
        for book in books:
            link_tag = book.find('h3').find('a')
            if link_tag and link_tag.get('href'):
                book_url = "http://books.toscrape.com/catalogue/" + link_tag['href'].replace('../../../', '')
                try:
                    book_data = get_book_data(book_url)
                    page_books_data.append(book_data)
                except Exception as e:
                    print(f"Ошибка при обработке книги {book_url}: {e}")

        print(f"Страница {page_num}: обработано {len(page_books_data)} книг")
        return page_books_data

    print("Начинаю парсинг книг...")
    scraping_start_time = time.time()

    # Парсим все страницы параллельно
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Создаем задачи для всех страниц
        future_to_page = {executor.submit(scrape_page, page): page for page in range(1, total_pages + 1)}

        # Собираем результаты по мере готовности
        for future in as_completed(future_to_page):
            page_books_data = future.result()
            all_books_data.extend(page_books_data)

    scraping_end_time = time.time()
    print(f"Парсинг завершен за {scraping_end_time - scraping_start_time:.2f} секунд")
    print(f"Итого книг: {len(all_books_data)}")

    # Сохранение в файл если is_save=True
    if is_save:
        filename = "books_data.txt"
        with open(filename, 'w', encoding='utf-8') as f:
            for book in all_books_data:
                f.write(str(book) + '\n')
        print(f"Данные сохранены в файл: {filename}")

    return all_books_data


# Проверка работы функции
if __name__ == "__main__":
    start_time = time.time()

    res = scrape_books(is_save=True)
    print(type(res), len(res))

    end_time = time.time()
    print(f"Общее время выполнения: {end_time - start_time:.2f} секунд")
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

Начинаю поиск количества страниц...
Страница 1: существует
Страница 2: существует
Страница 3: существует
Страница 4: существует
Страница 5: существует
Страница 6: существует
Страница 7: существует
Страница 8: существует
Страница 9: существует
Страница 10: существует
Страница 11: существует
Страница 12: существует
Страница 13: существует
Страница 14: существует
Страница 15: существует
Страница 16: существует
Страница 17: существует
Страница 18: существует
Страница 19: существует
Страница 20: существует
Страница 21: существует
Страница 22: существует
Страница 23: существует
Страница 24: существует
Страница 25: существует
Страница 26: существует
Страница 27: существует
Страница 28: существует
Страница 29: существует
Страница 30: существует
Страница 31: существует
Страница 32: существует
Страница 33: существует
Страница 34: существует
Страница 35: существует
Страница 36: существует
Страница 37: существует
Страница 38: существует
Страница 39: существует
Страница 40: существует
Страница 41: 

In [12]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

Начинаю поиск количества страниц...
Страница 1: существует
Страница 2: существует
Страница 3: существует
Страница 4: существует
Страница 5: существует
Страница 6: существует
Страница 7: существует
Страница 8: существует
Страница 9: существует
Страница 10: существует
Страница 11: существует
Страница 12: существует
Страница 13: существует
Страница 14: существует
Страница 15: существует
Страница 16: существует
Страница 17: существует
Страница 18: существует
Страница 19: существует
Страница 20: существует
Страница 21: существует
Страница 22: существует
Страница 23: существует
Страница 24: существует
Страница 25: существует
Страница 26: существует
Страница 27: существует
Страница 28: существует
Страница 29: существует
Страница 30: существует
Страница 31: существует
Страница 32: существует
Страница 33: существует
Страница 34: существует
Страница 35: существует
Страница 36: существует
Страница 37: существует
Страница 38: существует
Страница 39: существует
Страница 40: существует
Страница 41: 

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [13]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ
# Настройка планировщика задач
# Запускает функцию парсинга каждый день в 19:00 с сохранением данных в файл
schedule.every().day.at("19:00").do(lambda: scrape_books(is_save=True))
print("Планировщик запущен. Ожидание выполнения задач...")
# Основной цикл планировщика
# Проверяет наличие задач для выполнения каждые 60 секунд для оптимизации нагрузки
while True:
    schedule.run_pending()
    time.sleep(60)
# КОНЕЦ ВАШЕГО РЕШЕНИЯ

Планировщик запущен. Ожидание выполнения задач...
Начинаю поиск количества страниц...
Страница 1: существует
Страница 2: существует
Страница 3: существует
Страница 4: существует
Страница 5: существует
Страница 6: существует
Страница 7: существует
Страница 8: существует
Страница 9: существует
Страница 10: существует
Страница 11: существует
Страница 12: существует
Страница 13: существует
Страница 14: существует
Страница 15: существует
Страница 16: существует
Страница 17: существует
Страница 18: существует
Страница 19: существует
Страница 20: существует
Страница 21: существует
Страница 22: существует
Страница 23: существует
Страница 24: существует
Страница 25: существует
Страница 26: существует
Страница 27: существует
Страница 28: существует
Страница 29: существует
Страница 30: существует
Страница 31: существует
Страница 32: существует
Страница 33: существует
Страница 34: существует
Страница 35: существует
Страница 36: существует
Страница 37: существует
Страница 38: существует
Страница 39

KeyboardInterrupt: 

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [ ]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
# Не понял что в этой ячейке конкретно нужно оставить, тесты в проекте есть, в директории /tests
! pytest test/test_scraper.py

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```